In [3]:
import MisrToolkit as mtk

In [4]:
misr_file = '/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN_GM_P043_O117045_DF_F03_0024.hdf'

In [5]:
m = mtk.MtkFile(misr_file)


In [27]:
m.attr_get('Path_number')
r = mtk.MtkRegion(43,62,63)

In [ ]:
r.center
map_info = r.snap_to_grid(43, 1100)

In [ ]:
lat,lon = map_info.create_latlon()

In [ ]:
d.mapinfo()

In [7]:
m.attr_list

['HDFEOSVersion',
 'StructMetadata.0',
 'Path_number',
 'AGP_version_id',
 'DID_version_id',
 'Number_blocks',
 'Ocean_blocks_size',
 'Ocean_blocks.count',
 'Ocean_blocks.numbers',
 'SOM_parameters.som_ellipsoid.a',
 'SOM_parameters.som_ellipsoid.e2',
 'SOM_parameters.som_orbit.aprime',
 'SOM_parameters.som_orbit.eprime',
 'SOM_parameters.som_orbit.gama',
 'SOM_parameters.som_orbit.nrev',
 'SOM_parameters.som_orbit.ro',
 'SOM_parameters.som_orbit.i',
 'SOM_parameters.som_orbit.P2P1',
 'SOM_parameters.som_orbit.lambda0',
 'Origin_block.ulc.x',
 'Origin_block.ulc.y',
 'Origin_block.lrc.x',
 'Origin_block.lrc.y',
 'Start_block',
 'End block',
 'Cam_mode',
 'Num_local_modes',
 'Local_mode_site_name',
 'Orbit_QA',
 'Camera',
 'coremetadata']

In [8]:
m.block_metadata_list

['PerBlockMetadataCommon', 'PerBlockMetadataRad', 'PerBlockMetadataTime']

In [10]:
m.block_metadata_field_list('PerBlockMetadataTime')

['BlockCenterTime']

In [26]:
times = m.block_metadata_field_read('PerBlockMetadataTime','BlockCenterTime')
a = [ t for t in times if t is not '' ]
date = a[1]
date= pd.to_datetime(date).date()


datetime.date(2021, 12, 19)

In [16]:
m.grid_list

['BlueBand',
 'GreenBand',
 'RedBand',
 'NIRBand',
 'BRF Conversion Factors',
 'GeometricParameters']

In [28]:
mgrid =  m.grid('BlueBand')

In [29]:
mgrid.attr_list

['Block_size.resolution_x',
 'Block_size.resolution_y',
 'Block_size.size_x',
 'Block_size.size_y',
 'Scale factor',
 'std_solar_wgted_height',
 'SunDistanceAU']

In [30]:
res = mgrid.attr_get('Block_size.resolution_x')

In [31]:
res

1100

In [ ]:
#latlon_to_bls(path, resolution meters, lat, lon)
#Convert decimal degrees latitude and longitude to block, line, sample.

mtk.latlon_to_bls(path, res, 35.03588330130763, -121.10547839606753)

In [ ]:
import os
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
#import mpl_toolkits.basemap.pyproj as pyproj
import numpy as np
#from mpl_toolkits.basemap import Basemap
from pyhdf.SD import *
import MisrToolkit as mtk
import cv2
class MISR_Data:
    def __init__(self,misr_file,prefix = 'MISR_AM1_GRP_TERRAIN_GM_'):
        self.misr_file = mtk.MtkFile(misr_file) 
        self.prefix = prefix
        self.resolution = [128,512]
        self.date = self.get_date()
        self.start_block = int(misr.attr_get('Start_block'))
        self.end_block = int(misr.attr_get('End block'))
        self.blocks = np.arange(self.start_block,self.end_block,dtype='int')
        self.scale_factor = 1


    def get_date(self):
        m =  self.misr_file # ,mask_and_scale=True)
        times = m.block_metadata_field_read('PerBlockMetadataTime','BlockCenterTime')
        a = [ t for t in times if t is not '' ]
        date = a[1]
        date = pd.to_datetime(date).date()
        self.date = date
        return date


    def load(self):
        misr = self.misr_file # ,mask_and_scale=True)
        #self.hdf = SD(self.misr_file, SDC.READ)
        images = self.get_blocks_images()
        latitude, longitude = self.get_latitude_longitude()
        return images, latitude, longitude
    #("BlueBand|Blue Radiance/RDQI", "GreenBand|Green Radiance/RDQI",
    # "NIRBand|NIR Radiance/RDQI", "RedBand|Latitude", "RedBand|Longitude", "RedBand|Red Radiance/RDQI")

    def load_block(self,block_id):
        blocks_images_list, latitude, longitude = self.load()
        return blocks_images_list[block_id], latitude[block_id],longitude[block_id]

    def get_channel(self,channel_name,block_num,image_scale=1.):
        misr = self.misr_file
        #print('Reading channel ', channel_name)
        mgrid = misr.grid('BlueBand')
        path = misr.attr_get('Path_number')
        block_num = int(block_num)
        r = mtk.MtkRegion(path,block_num,block_num)
        
        data = data3D[block_num, :, :]
        attrs = data3D.attributes(full=1)
        fva = attrs["_FillValue"]
        _FillValue = fva[0]
        datas = np.right_shift(data, 2);
        dataf = datas.astype(np.double)
        dataf[data == _FillValue] = np.nan
        dataf[datas > 16376] = np.nan
        if dataf.shape[0] > self.resolution[0] :
            image_scale = 0.25
            dataf = cv2.resize(dataf,(0,0),fx = image_scale,fy= image_scale, interpolation=cv2.INTER_CUBIC)
        datam = np.ma.masked_array(dataf, mask=np.isnan(dataf))
        datam = self.scale_factor * datam;
        return datam

    def get_4d_image(self, block_num):
        channels = [ 'Red Radiance/RDQI', 'Green Radiance/RDQI','Blue Radiance/RDQI', 'NIR Radiance/RDQI']
        scale_factor = 1
        channels_list = []
        for channel_name in channels:
            ch = self.get_channel(channel_name,block_num)
            channels_list.append(ch)
        channels_im = np.ma.stack(channels_list,axis=0)
        return channels_im

    def get_blocks_images(self):
        blocks_images = []
        for block_num in self.blocks:
            block_im = self.get_4d_image(block_num)
            blocks_images.append(block_im)
        return blocks_images

    def parse_file_name(self):
        file_name_start = self.misr_file.find(self.prefix)
        data_to_parse = self.misr_file[file_name_start:]
        path_number = data_to_parse[1:4]
        orbit_number = data_to_parse[6:12]
        camera = data_to_parse[13:15]
        start_block_data = data_to_parse.find('.')
        block_data_to_parse = data_to_parse[start_block_data:]
        self.start_block = block_data_to_parse[1:4]
        self.end_block = block_data_to_parse[5:8]

    def get_latitude_longitude(self):
        blocks_latitude = []
        blocks_longitude = []
        lat = self.hdf.select('Latitude')
        lon = self.hdf.select('Longitude')
        for block_num in self.blocks:
            block_num = int(block_num)
            block_lat = lat[block_num,:,:]
            block_lon = lon[block_num,:,:]
            blocks_latitude.append(block_lat)
            blocks_longitude.append(block_lon)
        return blocks_latitude,blocks_longitude

    def get_max_min_lon_lat_per_block(self):
        #change this after understaning how to read it crrectly
        #misr = rxr.open_rasterio(self.misr_file)  # ,mask_and_scale=True)

        images , latitude, longitude = self.load()
        lat_lon_ranges_all_blocks = []
        for block_lat,block_lon in zip(latitude,longitude):
            lat_lon_ranges = {}
            lat_lon_ranges['min_lat'] = float(np.min(block_lat))
            lat_lon_ranges['max_lat'] = float(np.max(block_lat))
            lat_lon_ranges['min_lon'] = float(np.min(block_lon))
            lat_lon_ranges['max_lon'] = float(np.max(block_lon))
            lat_lon_ranges_all_blocks.append(lat_lon_ranges)
        return lat_lon_ranges_all_blocks

    def get_orbit(self):
        misr = rxr.open_rasterio(self.misr_file)  # ,mask_and_scale=True)
        try:
            attrs = misr[0].attrs
            orbit = attrs['ORBITNUMBER.1']

        except:
            attrs = misr.attrs
            orbit = attrs['ORBITNUMBER.1']

        return orbit


In [20]:
!pip install pandas

     |████████████████████████████████| 9.5 MB 899 kB/s            


In [35]:
import pandas as pd
a = "2021-12-19T19:24:52.195362Z"
print(pd.to_datetime(a).date())

2021-12-19


In [39]:
str(pd.to_datetime(a).date())

'2021-12-19'

# Create Pandas array from misr data



In [61]:
import os
import re
#import matplotlib as mpl
#import matplotlib.pyplot as plt
#import mpl_toolkits.basemap.pyproj as pyproj
import numpy as np
#from mpl_toolkits.basemap import Basemap
#from pyhdf.SD import *
#import xarray as xr
#import rioxarray as rxr
#import cv2
import pandas
import MisrToolkit as mtk
class MISR_tabular_data:
    def __init__(self,file):
        self.misr_file = mtk.MtkFile(file)
    
    def get_date(self):
        m =  self.misr_file # ,mask_and_scale=True)
        times = m.block_metadata_field_read('PerBlockMetadataTime','BlockCenterTime')
        a = [ t for t in times if t is not '' ]
        date = a[1]
        date = pandas.to_datetime(date).date()
        self.date = str(date)
        #m.core_metadata_get (EQUATORCROSSINGTIME)
        return str(date)
    
    def get_path(self):
        m =  self.misr_file 
        path = m.attr_get('Path_number')
        return path
    
    def get_orbit(self):
        m =  self.misr_file 
        orbit = m.core_metadata_get('ORBITNUMBER')
        return orbit
    
    def get_camera(self):
        m =  self.misr_file
        cam = m.attr_get('Camera')
        return cam

    def check_if_pm_station_is_in_misr_image(self,pm_lat,pm_lon):
        m = self.misr_file
        path = m.attr_get('Path_number')
        start_block = int(m.attr_get('Start_block'))
        end_block = int(m.attr_get('End block'))
        blocks = np.arange(start_block,end_block,dtype='int')
        #r = mtk.MtkRegion(path,start_block,end_block)
        mgrid =  m.grid('BlueBand')
        res = mgrid.attr_get('Block_size.resolution_x')
        #map_info = r.snap_to_grid(path, res)
        #lat,lon = map_info.create_latlon()
        try:
            mtk.latlon_to_bls(path,res,pm_lat, pm_lon)
        except:
            return False
        
        return True

In [67]:
def get_misr_data_df(misr_dir):
    misr_files = [os.path.join(misr_subset_dir,f) for f in os.listdir(misr_subset_dir) if f.endswith('hdf')]
    all_data = []
    for i, file in enumerate(misr_files):
        misr = MISR_tabular_data(file)
        d = {'file_name': file ,'date' : misr.get_date() , 'orbit': misr.get_orbit(), 'path' : misr.get_path(), 'camera' : misr.get_camera() }
        all_data.append(d)

    df = pandas.DataFrame(all_data)
    return df

In [68]:
misr_dir = '/home/adiraz/adiraz/misr_data'
misr_df = get_misr_data_df(misr_dir)
misr_df

,file_name,date,orbit,path,camera
0,/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TER...,2021-12-30,117205,40,9
1,/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TER...,2021-12-30,117205,40,8
2,/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TER...,2021-12-30,117205,40,7
3,/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TER...,2021-12-30,117205,40,6
4,/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TER...,2021-12-30,117205,40,5
...,...,...,...,...,...
235,/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TER...,2021-10-09,116011,42,8
236,/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TER...,2021-10-09,116011,42,7
237,/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TER...,2021-10-09,116011,42,6
238,/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TER...,2021-10-09,116011,42,5


In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from pm_data import PM_Data
from misr_tabular_data import get_misr_data_df
import pandas
pm_data_file = '/home/adiraz/adiraz/misr_data/la_pm_2021_daily.json'
misr_dir = '/home/adiraz/adiraz/misr_data'

pdaily = PM_Data(pm_data_file)
pm_data = pdaily.load()
misr_df = get_misr_data_df(misr_dir)


In [51]:
file

'/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN_GM_P042_O116011_AF_F03_0024.hdf'